# top:

In [40]:
import pandas as pd
import sqldf

# more handcoded:

In [5]:
bw=pd.read_csv("bw0731.csv")
handcoded=pd.read_excel("../w8/BWID_factoryname handcoded.xlsx", sheet_name='data',keep_default_na=False, na_values=[""])
handcodedIdx=handcoded[handcoded.RRCleaned.isna()==False].FactoryBWID.unique()
ci=pd.read_csv("ci_w9.csv")

In [16]:
bw.loc[bw.FactoryBWID.isin(handcodedIdx),'handcoded']=1.0

# branch of ...

In [23]:
# bw[bw.FactoryName.str.lower().str.contains("branch of")]
bw[bw.cName.str.contains("branch of")]

,QuestID,FactoryBWID,FactoryName,Country,AssessmentDate,year,cName,CNRR,repriskid,RRCleaned,Rrsector,doubt,handcoded,round


In [22]:
bw.loc[6453,"cName"]='viet tien garment corp'

# \#RRCleaned > \#CNRR/repriskID:

In [92]:
# for i in bw.RRCleaned.unique():
#     if i not in ci.cName.unique():
#         print(i)

bw[(bw.RRCleaned.isna()==False)&(bw.CNRR.isna()==True)].head(1) # CNRR is None, not NaN

,QuestID,FactoryBWID,FactoryName,Country,AssessmentDate,year,cName,CNRR,repriskid,RRCleaned,Rrsector,doubt,handcoded,round
423,11228,169,New Wide Garment (Viet Nam) Co. Ltd.,Vietnam,20/10/2015,2015,new wide garment,None,NaN,new wide garment,Personal and Household Goods,0.0,0.0,1.0


In [103]:
for i in bw[bw.RRCleaned.isna()==False].RRCleaned.unique():
    if bw.loc[bw.RRCleaned==i,"CNRR"].isna().any():
    #  or bw.loc[bw.repriskid==i,"CNRR"].to_list()==[]:
        # print(i,1)
        bw.loc[bw.RRCleaned==i,"CNRR"]=ci.loc[ci.cName==i,'company_name'].values[0]
    if bw.loc[bw.RRCleaned==i,"Rrsector"].isna().any():
        # or bw.loc[bw.repriskid==i,"Rrsector"].to_list()==[]:
        # print(i,2)
        bw.loc[bw.RRCleaned==i,"Rrsector"]=ci.loc[ci.cName==i,'sectors'].values[0]
    if bw.loc[bw.RRCleaned==i,"repriskid"].isna().any():
        #  or bw.loc[bw.repriskid==i,"repriskid"].to_list()==[]:
        # print(i,3)
        bw.loc[bw.RRCleaned==i,"repriskid"]=ci.loc[ci.cName==i,'repriskID'].values[0]

In [107]:
bw[["CNRR",	"repriskid"	,"RRCleaned",	"Rrsector"]].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6500 entries, 0 to 6499
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   CNRR       2050 non-null   object 
 1   repriskid  2050 non-null   float64
 2   RRCleaned  2050 non-null   object 
 3   Rrsector   2050 non-null   object 
dtypes: float64(1), object(3)
memory usage: 253.9+ KB


---

In [37]:
for i in bw[bw.repriskid.isna()==False].repriskid.unique():
    if bw.loc[bw.repriskid==i,"CNRR"].isna().any():
        bw.loc[bw.repriskid==i,"CNRR"]=ci.loc[ci.repriskID==i,'company_name'].values[0]
    if bw.loc[bw.repriskid==i,"Rrsector"].isna().any():
        bw.loc[bw.repriskid==i,"Rrsector"]=ci.loc[ci.repriskID==i,'sectors'].values[0]
    if bw.loc[bw.repriskid==i,"RRCleaned"].isna().any():
        bw.loc[bw.repriskid==i,"RRCleaned"]=ci.loc[ci.repriskID==i,'cName'].values[0]

# sqldf.run('select distinct repriskid from bw group by repriskid having  count( distinct CNRR)>1')

In [64]:
# bw[bw.RRCleaned.isin(["Thuan Phuong Embroideries Garments Co Ltd",
# "Hanoi Textile and Garment JSC",
# "Dap Cau Garment Corp JSC"])]

bw[bw.FactoryBWID.isin([2398,2491,3116])]

,QuestID,FactoryBWID,FactoryName,Country,AssessmentDate,year,cName,CNRR,repriskid,RRCleaned,Rrsector,doubt,handcoded,round
5427,24969,2398,Branch of Thuan Phuong Embroideries Garments C...,Vietnam,30/01/2018,2018,thuan phuong embroideries garments,Thuan Phuong Embroideries Garments Co Ltd (Thu...,2232388.0,thuan phuong embroideries garments,Personal and Household Goods,0.0,0.0,1.0
5428,31854,2398,Branch of Thuan Phuong Embroideries Garments C...,Vietnam,15/01/2019,2019,thuan phuong embroideries garments,Thuan Phuong Embroideries Garments Co Ltd (Thu...,2232388.0,thuan phuong embroideries garments,Personal and Household Goods,0.0,0.0,1.0
5429,38985,2398,Branch of Thuan Phuong Embroideries Garments C...,Vietnam,26/02/2020,2020,thuan phuong embroideries garments,Thuan Phuong Embroideries Garments Co Ltd (Thu...,2232388.0,thuan phuong embroideries garments,Personal and Household Goods,0.0,0.0,1.0
5430,41246,2398,Branch of Thuan Phuong Embroideries Garments C...,Vietnam,26/01/2021,2021,thuan phuong embroideries garments,Thuan Phuong Embroideries Garments Co Ltd (Thu...,2232388.0,thuan phuong embroideries garments,Personal and Household Goods,0.0,0.0,1.0
5542,23552,2491,Branch of Hanoi Textile and Garment JSC,Vietnam,14/11/2017,2017,hanoi textile and garment jsc,Hanoi Textile and Garment Joint Stock Corp (Vi...,125560.0,hanoi textile and garment jsc,None,0.0,1.0,0.0
5543,31733,2491,Branch of Hanoi Textile and Garment JSC,Vietnam,8/1/2019,2019,hanoi textile and garment jsc,Hanoi Textile and Garment Joint Stock Corp (Vi...,125560.0,hanoi textile and garment jsc,None,0.0,1.0,0.0
5544,39231,2491,Branch of Hanoi Textile and Garment JSC,Vietnam,24/03/2020,2020,hanoi textile and garment jsc,Hanoi Textile and Garment Joint Stock Corp (Vi...,125560.0,hanoi textile and garment jsc,None,0.0,1.0,0.0
5545,41849,2491,Branch of Hanoi Textile and Garment JSC,Vietnam,24/03/2021,2021,hanoi textile and garment jsc,Hanoi Textile and Garment Joint Stock Corp (Vi...,125560.0,hanoi textile and garment jsc,None,0.0,1.0,0.0
6312,36891,3116,Branch of Dap Cau Garment Corp. JSC - Kinh Bac...,Vietnam,29/10/2019,2019,dap cau garment corp jsc,Dap Cau Garment Corp JSC,2386448.0,dap cau garment corp jsc,Personal and Household Goods,0.0,1.0,NaN
6313,40632,3116,Branch of Dap Cau Garment Corp. JSC - Kinh Bac...,Vietnam,5/11/2020,2020,dap cau garment corp jsc,Dap Cau Garment Corp JSC,2386448.0,dap cau garment corp jsc,Personal and Household Goods,0.0,1.0,0.0


# correction:

In [63]:
correction={'Thuan Phuong Embroideries Garments Co Ltd':'thuan phuong embroideries garments',
'Hanoi Textile and Garment JSC':'hanoi textile and garment jsc',
'Dap Cau Garment Corp JSC':'dap cau garment corp jsc'}
for i in correction:
    if bw.loc[bw.RRCleaned==i, "repriskid"].isna().any():
        bw.loc[bw.RRCleaned==i, "repriskid"]=ci.loc[ci.cName==correction[i],'repriskID'].values[0]
    bw.loc[bw.RRCleaned==i, "RRCleaned"]=correction[i]

# export

In [105]:
bw.to_csv("bw0801.csv",index=False)